In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd



In [ ]:
#export
def download_files_maps():
    route = "./local/data/"
    gdd.download_file_from_google_drive(file_id='1wEiDe2B_ojoaLUnV3Ik67t_YpAC2nbPA',
                                        dest_path=route+"colombia.geo.json",
                                        unzip=False, showsize=True, overwrite = True)
    return 

In [ ]:
download_files_maps()

In [ ]:
def download_files_dengue():
    route = "./local/data/"
    gdd.download_file_from_google_drive(file_id='1xHXshzID1MBrKZKVvnc6JeO_ZU04HvQ_',
                                        dest_path=route+"dengue2007-2017.csv",
                                        unzip=False, showsize=True, overwrite = True)
    return
    

In [ ]:
download_files_dengue()

65.2 MiB                                                                                                                                                                                                                                                                                                                                                           Done.
